<a href="https://colab.research.google.com/github/Yeonseo-Jo/Class2022Spring/blob/main/(14)%20huggingface_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio
import gradio as gr          #gradio import

In [4]:
#한 줄로 끝나는 gradio도 있음 (input,output,function 입력 없이 제목만 입력해도 되는 경우)

gr.Interface.load("huggingface/gpt2").launch()

Fetching model from: https://huggingface.co/gpt2
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://27573.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7ff73173dd50>,
 'http://127.0.0.1:7860/',
 'https://27573.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


### [Image classification](https://huggingface.co/tasks/image-classification)

* Computer Vision의 한 종류
* image classification = 1000개의 class 중에 어떤 class와 가까운지 그 확률을 보여줌

e.g. https://huggingface.co/google/vit-base-patch16-224 \
How to use

* image classification 중 하나를 선택해서 가져옴
* how to use에 들어가서 사용법 보기

In [ ]:
!pip install transformers

In [5]:
from transformers import ViTFeatureExtractor, ViTForImageClassification
from PIL import Image             #지난번에는 open cv를 이용했지만 이번엔 PIL을 이용해서 이미지를 가져올 것!
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)                      #이미지는 숫자 형식 (ex. color: 3d array / 흑백: 1d array)

feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')         #FeatureExtractor를 할 수 있는 모델임
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')         #Imageclassification을 할 수 있는 모델임 (FeatureExtractor 기반으로)

inputs = feature_extractor(images=image, return_tensors="pt")             #feautre_extractor = input으로 들어갈 image의 feature를 압축해서 넣는 기능을 함 
#len(image) > len(inputs) => image에서 정보를 뽑아낸 것이 input이니까      =많은 숫자 정보로부터 image의 limited 된 숫자 정보를 뽑아내는 기능
outputs = model(**inputs)                                                 # output에는 classify된 정보가 들어감ㅁ
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes                 # feature로부터 1000개의 class가 뽑아지고, 그 1000개에 대한 확률 정보가 나오는 모델
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

#정리하자면, image classification은 input으로 이미지를 넣으면 class, class에 대한 확률을 ouput으로 뽑아내주는 모델임


# -> script 형태로 쓴 코드

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Predicted class: Egyptian cat


Demo in Gradio

* 위에서 가져온 image classification을 gradio로 만들기

In [9]:
#위에서 쓴 script 형태의 코드를 함수의 형태로 바꿔줌

def func (image):
  feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
  model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

  inputs = feature_extractor(images=image, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  # model predicts one of the 1000 ImageNet classes
  predicted_class_idx = logits.argmax(-1).item()
  predicted_class = model.config.id2label[predicted_class_idx]
  return predicted_class

  #gradio로 만들기 위해서 function화하기
  #image가 input으로 들어오면, 위에서 가져온 model을 거치게 하는 function

In [10]:
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/tiger.jpg"
os.system("curl " + url + " > tiger.jpg")
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/dog.jpg"
os.system("curl " + url + " > dog.jpg")    

0

In [11]:
#gradio로 만드는 과정

gr.Interface(fn=func, inputs='image', outputs='text', examples = ['tiger.jpg', 'dog.jpg']).launch()

#fn= 사용할 function 정의
#input = image => input 유형 정의
#output = text => output 유형 정정
#examples = gradio에 넣어볼 예제들 (위에서 url로 서버에 가져온 그림들) => 중요! 하나 이상이면 반드시 리스트 구조로 넣기


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://47764.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7ff5fd0b3610>,
 'http://127.0.0.1:7861/',
 'https://47764.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


### [Fill-Mask](https://huggingface.co/tasks/fill-mask)


* NLP의 한 종류
* 문장에서 [mask]에 해당하는 부분에 들어 올 수 있는 word들을 확률과 함께 보여줌

* Masked language modeling is the task of masking some of the words in a sentence and predicting which words should replace those masks. These models are useful when we want to get a statistical understanding of the language in which the model is trained in.


e.g. https://huggingface.co/bert-base-uncased \
How to use

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

Demo in Gradio

In [16]:
import pandas as pd
def func (text):
  unmasker = pipeline('fill-mask', model='bert-base-uncased')            #model을 불러와 unmask 변수에 저장
  result = unmasker(text)
  df = pd.DataFrame(result)
  return df                             #model을 dataframe화시킴

In [17]:
examples = ["Hello I'm a [MASK] model.", "It is raining outside. I feel [MASK]."]         #examples 생성

In [18]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://22400.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7ff5f72bd610>,
 'http://127.0.0.1:7862/',
 'https://22400.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


### [Token classification](https://huggingface.co/tasks/token-classification)

* NLP의 한 종류 = natural language understanding (natural language 중 필요한 정보를 뽑아내는 것)
 (1) NER(Named Entity Recognition) : 더 중요한 명사들을 catch 하는것 / 명사들이 어떤 정보를 나타내는지, 명사들의 종류=class를 알려줌.
    - by location(LOC), organizations (ORG), person (PER) and Miscellaneous (MISC)
 
 (2) POS(Part of Speech) : 품사 태깅


* Token classification is a natural language understanding task in which a label is assigned to some tokens in a text. Some popular token classification subtasks are Named Entity Recognition (NER) and Part-of-Speech (PoS) tagging. NER models could be trained to identify specific entities in a text, such as dates, individuals and places; and PoS tagging would identify, for example, which words in a text are verbs, nouns, and punctuation marks.



e.g. https://huggingface.co/dslim/bert-base-NER \
How to use

In [ ]:
!pip install transformers

NER 모델

=
recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

Demo in Gradio

In [24]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
def func (text):
  tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")                   #AI 모델을 불러와 tokenize 전처리를 해 줌
  model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")      #model은 각각의 단어들이 어떤 class인지 classify함
  nlp = pipeline("ner", model=model, tokenizer=tokenizer)
  result = nlp(text)
  df = pd.DataFrame(result)
  return df                        #출력이 리스트로 여러개 들어 있는 형태이므로 그 출력을 gradio에서는 dataframe화 할 수 있음

In [22]:
examples = ["My name is Wolfgang and I live in Berlin", "I will visit Seoul to see Chris"]

In [23]:
gr.Interface(fn=func, inputs='text', outputs='dataframe', examples = examples).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://37148.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7ff5f41248d0>,
 'http://127.0.0.1:7863/',
 'https://37148.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable


### [Sentence similarity](https://huggingface.co/tasks/sentence-similarity)

* 두 주어진 문장들이 얼만큼 유사한지 나타내줌

* Sentence Similarity is the task of determining how similar two texts are. Sentence similarity models convert input texts into vectors (embeddings) that capture semantic information and calculate how close (similar) they are between them. This task is particularly useful for information retrieval and clustering/grouping.

<br/>

* <b><u> 중요! text, sounds, image 등의 raw data들은 model에 넣기 위해서는 feature (=vector = embeddings)의 형태로 변환해야 함! </b> </u>


e.g. https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 \
How to use

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
sentences = ["This is an example sentence", "it is one example writing"]

model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
embeddings = model.encode(sentences)                    
print(embeddings)

* embeddings의 형태 -> 중첩 리스트

* len(embdeeings) = 2
* len(emeddings[0]) , len(embeddings[1]) = 384 (한 vector의 길이가 384개이다=> <u>각 feature들의 길이</u>)
 
 => embeddings는 384개를 갖는 리스트 2개가 중첩된 중첩리스트 형태임 => 2개의 vector가 중첩된 형태

</br>
* 두 text의 유사도는 각 text들의 각도 차이를 통해 계산함.

 => Theta = 0이면 유사도가 가장 높으며 Theta=90이면 유사도가 가장 낮음
 => cos(theta)=1이면(->cos(0), 즉 theta가 0일 때) 유사도가 가장 높고 cos(theta)=0이면(->cos(90), 즉 theta가 90일 때) 유사도가 가장 낮음

 * <b><u> 정리: 두 vector(=embeddings)의 similarity는 두 vector들의 각도값에 cosine(theta)를 해주면 됨. => cosine similarilty 로 두 vector들의 유사도를 계산한다 

 = 장점: cosine 각도를 하면 아무리 차원이 높아져도 원 점과 두 vector가 삼각형을 이룸 </b></u>

In [27]:
len(embeddings[1])

#len(embeddings) = 2 (리스트가 2개 중첩된 중첩 리스트 구조이므로)

384

In [28]:
cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])      
#cosine_scores: 두 vector간 거리(=각도)를 cosine 값을 취해측정해 그 유사도를 나타냄
cosine_scores

tensor([[0.5726]])

Demo in Gradio

In [31]:
#function화 시키기

def func (text1, text2):
  from sentence_transformers import SentenceTransformer, util
  model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
  embeddings = model.encode([text1, text2])
  cosine_scores = util.pytorch_cos_sim(embeddings[0], embeddings[1])
  return cosine_scores

In [32]:
examples = [["This is an example sentence", "it is one example writing"], ["A frog is hopping near the pond", "I love Korean Food"]]

In [33]:
gr.Interface(fn=func, inputs=['text', 'text'], outputs='number', examples = examples).launch()

#input =  text 두 개 받음 => 각각의 vector(=feature = embedding)가 됨

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://47458.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<gradio.routes.App at 0x7ff5edae5f90>,
 'http://127.0.0.1:7864/',
 'https://47458.gradio.app')

Exception in callback None(<Task finishe...> result=None>)
handle: <Handle>
Traceback (most recent call last):
  File "/usr/lib/python3.7/asyncio/events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
TypeError: 'NoneType' object is not callable
